<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/risk/correlations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from pandas_datareader import DataReader as pdr
import ssl

# Get return data from Aswath Damodoran
ssl._create_default_https_context = ssl._create_unverified_context
file = "http://www.stern.nyu.edu/~adamodar/pc/datasets/histretSP.xls"
sheet = "Returns by year"
df = pd.read_excel(file, sheet_name=sheet, skiprows=16, header=[0, 1])
col1 = ("Unnamed: 0_level_0", "Year")
indx = df[df[col1].isna()].index[0]
df = df.iloc[:indx].set_index(col1)
df.index.name = "Year"
df = df["Annual Returns on Investments in"]
df = df[df.columns[:4]]
df.columns = ["SP500", "TBills", "TBonds", "Corporates"]
df.dtype = "float"
df.index = [int(x) for x in df.index]
df.reset_index(inplace=True)
df = df.rename(columns={"index": "Year"})

# Get inflation data from Federal Reserve Economic Data
df_in = pdr("CPIAUCSL", "fred", start="1929-12-01")
df_in = df_in.resample("Y").last().iloc[:-1]
df_in = df_in.pct_change().reset_index()
df_in.columns = ["Year", "Inflation"]
df_in["Year"] = df_in.Year.map(lambda x: x.year)

# Merge data
df = df_in.merge(df, on="Year")
df = df.set_index('Year')

df

/Users/kailiao/PycharmProjects/appcode_draft/venv/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


,Inflation,SP500,TBills,TBonds,Corporates
Year,,,,,
1947,NaN,0.052000,0.006008,0.009197,0.002621
1948,0.027339,0.057046,0.010450,0.019510,0.034370
1949,-0.018295,0.183032,0.011150,0.046635,0.053773
1950,0.058026,0.308055,0.012033,0.004296,0.042388
1951,0.059648,0.236785,0.015175,-0.002953,-0.001910
...,...,...,...,...,...
2017,0.021299,0.216055,0.009308,0.028017,0.097239
2018,0.019152,-0.042269,0.019392,-0.000167,-0.027626
2019,0.022617,0.312117,0.015500,0.096356,0.153295


In [ ]:
df.corr()

,Inflation,SP500,TBills,TBonds,Corporates
Inflation,1.000000,-0.133546,0.658949,-0.122190,-0.161914
SP500,-0.133546,1.000000,-0.081840,-0.067505,0.352723
TBills,0.658949,-0.081840,1.000000,0.261221,0.151863
TBonds,-0.122190,-0.067505,0.261221,1.000000,0.622999
Corporates,-0.161914,0.352723,0.151863,0.622999,1.000000
